# Práctica 2: Limpieza y análisis de datos
## Autores
Hemos realizado esta práctica:
* Ignacio Such Ballester
* Andrés Isidro Fonts Santana

## 1. Descripción del _dataset_
### 1.1 Contexto
Se pretende sacar al mercado un nuevo juego de mesa lo más existoso posible y convertirlo en un bestseller.

Para ello, hemos escogido el _dataset_ [Board Game Data](https://www.kaggle.com/datasets/mrpantherson/board-game-data?select=bgg_db_2018_01.csv), disponible en la plataforma Kaggle.

Este conjunto de datos se ha extraído mediante la API del portal [Board Games Geek](https://boardgamegeek.com/). El _dataset_ se generó en enero de 2018 y contiene datos sobre los primeros 5000 juegos de mesa del _ranking_ de Board Games Geek. 

A través de este set de datos, podemos realizar un análisis profundo del mismo, obteniendo correlaciones, clasificaciones en incluso predicciones para averigurar cómo diseñar nuestro juego de mesa.

Además, se podrá proceder a crear modelos de regresión que permitan predecir si un juego será un bestseller o no en función de sus características y contrastes de hipótesis que ayuden a identificar propiedades interesantes en las muestras.

### 1.2 Descripción de los atributos 
Cada uno de los 5000 registros con que cuenta al _dataset_ viene determinado por 20 attributos:

| Nombre      | Tipo    | Descripción                         | Ejemplo
|:------------|:--------|:------------------------------------|:-------------------------------------------------------------
| rank        | int     | Posición en el _ranking_ de BGG     | 21
| bgg_url     | string  | Link a url de la reseña en BGG      | https://boardgamegeek.com/boardgame/167791/terraforming-mars
| game_id     | string  | Identificador del juego en BGG      | 25613
| names       | string  | Nombre del juego                    | Terraforming Mars
| min_players | int     | Número mínimo de jugadores          | 2
| max_players | int     | Número máximo de jugadores          | 4
| avg_time    | int     | Tiempo medio de partida (minutos)   | 60
| min_time    | int     | Tiempo mínimo de partida (minutos)  | 30
| max_time    | int     | Tiempo máx de partida (minutos)     | 120
| year        | int     | Año de publicación                  | 2014
| avg_rating  | float   | Puntuación media del juego según usuarios de BGG (sobre 10)| 8.0096
| geek_rating | float   | Puntuació de BGG, obtenida a través de un algoritmo de ponderación bayesiana (algoritmo no público) |8.49837
| num_votes   | int     | Número de usuarios que han dado puntuación al juego | 1779
| image_url   | string  | Enlace a la imagen del juego  | https://cf.geekdo-images.com/images/pic361592.jpg
| age         | int     | Edad mínimia recomendada | 12
| mechanic    | string  | Tipo de Mecánicas del juego, separadas por comas | Area Enclosure, Card Drafting, Hand Management, Variable Player Powers, Worker Placement
| owned       | int     | Número de usuarios de BGG que han notificado que poseen el juego | 18217
| category    | string  | Categorías a las que pertenece el juego, separadas por comas | Ancient, Card Game, City Building, Civilization
| designer    | string  | Diseñador/a del juego. Si más de uno, separados por comas | Jamey Stegmaier
| weight      | float   | Grado de complejidad del juego (sobre 5) | 2.394


## 2. Carga de datos y selección
Utilizaremos la librería `pandas` para trabajar con los datos.

In [79]:
import pandas as pd
from IPython.display import display, HTML
import matplotlib

# Llamamos "bgg" al dataframe creado a partir del dataset
bgg = pd.read_csv('../csv/bgg_db_2018_01.csv',sep=',',encoding='latin-1')

Constatamos que los atributos `bgg_url` y `image_url` no nos van a aportar ningún matiz ni información relevante al estudio que queremos realizar sobre los datos. 

Eliminamos estos datos usando el método `.drop()`. 

In [80]:
# Eliminamos las columnas que no utilizaremos para el análisis
bgg.drop('bgg_url'  , inplace=True, axis=1)
bgg.drop('image_url', inplace=True, axis=1)

Por otro lado, según la documentación de BoardGameGeek, para evaluar el _ranking_ global del juego se utiliza el atributo `geek_rating`. Este atributo realiza una ponderación Bayesiana utilizando un algoritmo que BGG no publica.

Sí sabemos que en este algoritmo se compensan aquellos juegos con pocas valoraciones, pudiendo añadir hasta 100 votos _virtuales_ (ver [este enlace](https://boardgamegeek.com/wiki/page/BoardGameGeek_FAQ#toc13)).

Para nuestro estudio, partiremos de aquellos datos que tengan como mínimo 500 valoracion de usuarios.

In [81]:
# Filtrar por mínimo de votos
bgg = bgg[bgg.num_votes > 500]

## 3. Limpieza de los datos

**NOTAS**

Ojo, he visto que avg_time es equivalente a max_time, por lo que habría que recalcular este parámetro como (max-min)/2

Por otro lado, en el dataset no hay valores "NA", pero no quiere decir que no haya valores nulos. Mirando los datos el valor '0' parece indicar el valor nulo o vacío. Hay que tratar esos casos y en vez de poner '0' que ponga 'null' o similar.

También hay al menos 2 entradas que no tienen sentido, como son "Unpublished Prototype" y "Out of the scope of BGG". Fíjate que cuando buscas qué registros tienen valores vacíos te aparecen. Habría que retirarlos del dataset.

Finalmente, sí podemos tener juegos con duración muy extendida, en caso de ser un "Legacy", en que el juego se va desarrollando en modo campaña y se tardan decenas de partidas en acabar el juego. Este tipo de juegos típicamente no tiene rejugabilidad.


In [82]:
# Show 5 rows of the dataframe
bgg.head()

,rank,game_id,names,min_players,max_players,avg_time,min_time,max_time,year,avg_rating,geek_rating,num_votes,age,mechanic,owned,category,designer,weight
0,1,174430,Gloomhaven,1,4,150,90,150,2017,9.01310,8.52234,9841,12,"Action / Movement Programming, Co-operative Pl...",18217,"Adventure, Exploration, Fantasy, Fighting, Min...",Isaac Childres,3.7720
1,2,161936,Pandemic Legacy: Season 1,2,4,60,60,60,2015,8.66575,8.49837,23489,13,"Action Point Allowance System, Co-operative Pl...",38105,"Environmental, Medical","Rob Daviau, Matt Leacock",2.8056
2,3,182028,Through the Ages: A New Story of Civilization,2,4,240,180,240,2015,8.65702,8.32401,10679,14,"Action Point Allowance System, Auction/Bidding...",14147,"Card Game, Civilization, Economic",Vlaada Chvátil,4.3538
3,4,12333,Twilight Struggle,2,2,180,120,180,2005,8.35188,8.21012,29923,13,"Area Control / Area Influence, Campaign / Batt...",41094,"Modern Warfare, Political, Wargame","Ananda Gupta, Jason Matthews",3.5446
4,5,167791,Terraforming Mars,1,5,120,120,120,2016,8.38331,8.17328,20468,12,"Card Drafting, Hand Management, Tile Placement...",26145,"Economic, Environmental, Industry / Manufactur...",Jacob Fryxelius,3.2465


In [83]:
# Show number of rows in the dataframe
bgg.shape

(2721, 18)

In [84]:
# Show if dataframes has NA values
bgg.isnull().sum()

# Vemos que en el dataset no existen valores nulos

# Show 0 values of the dataframe

rank           0
game_id        0
names          0
min_players    0
max_players    0
avg_time       0
min_time       0
max_time       0
year           0
avg_rating     0
geek_rating    0
num_votes      0
age            0
mechanic       0
owned          0
category       0
designer       0
weight         0
dtype: int64

In [85]:
# Show extreme values with 2 decimals
bgg.describe().round(2)

# Aquí se puede ver que para algunas observaciones, encontramos juegos que tienen valores extremos, como son en las variables avg_time, min_time y max_time. También para la variable max_players, se encuentran valores extremos, como son en el caso de que el valor sea 0.

,rank,game_id,min_players,max_players,avg_time,min_time,max_time,year,avg_rating,geek_rating,num_votes,age,owned,weight
count,2721.00,2721.00,2721.00,2721.00,2721.00,2721.00,2721.00,2721.00,2721.00,2721.00,2721.00,2721.00,2721.00,2721.00
mean,1541.47,76880.33,2.07,5.21,86.48,67.53,86.33,1996.08,6.92,6.35,3267.92,10.51,4794.14,2.29
std,1078.85,71001.19,0.69,6.16,294.46,142.69,294.49,161.60,0.55,0.50,5775.83,2.73,7811.86,0.77
min,1.00,1.00,0.00,0.00,0.00,0.00,0.00,-3000.00,5.77,5.64,501.00,0.00,458.00,0.00
25%,681.00,8668.00,2.00,4.00,30.00,30.00,30.00,2003.00,6.50,5.96,798.00,8.00,1444.00,1.71
50%,1366.00,45986.00,2.00,4.00,60.00,45.00,60.00,2009.00,6.88,6.24,1394.00,10.00,2365.00,2.23
75%,2205.00,144270.00,2.00,6.00,90.00,75.00,90.00,2014.00,7.31,6.65,3075.00,12.00,4695.00,2.81
max,4987.00,236191.00,8.00,99.00,12000.00,6000.00,12000.00,2017.00,9.07,8.52,74261.00,18.00,106608.00,4.70


In [86]:
# Show row where is max value of avg_time
bgg[bgg['avg_time']==bgg['avg_time'].max()]

,rank,game_id,names,min_players,max_players,avg_time,min_time,max_time,year,avg_rating,geek_rating,num_votes,age,mechanic,owned,category,designer,weight
930,930,254,Empires in Arms,2,7,12000,6000,12000,1983,7.53768,6.46514,1099,14,"Area Movement, Dice Rolling, Secret Unit Deplo...",1851,"Economic, Napoleonic, Negotiation, Political, ...","Greg Pinder, Harry Rowland",4.4316


In [87]:
# Show rows where max_players equals 0
bgg[bgg['max_time']==0]

,rank,game_id,names,min_players,max_players,avg_time,min_time,max_time,year,avg_rating,geek_rating,num_votes,age,mechanic,owned,category,designer,weight
790,791,177524,Ice Cool,2,4,20,20,0,2016,6.93164,6.55288,3301,6,"Area Movement, Take That",5181,"Action / Dexterity, Animals, Children's Game",Brian Gomez,1.0000
799,800,160902,Dungeons & Dragons Dice Masters: Battle for Fa...,2,2,60,60,0,2015,7.42356,6.54724,1265,14,"Deck / Pool Building, Dice Rolling, Variable P...",3295,"Collectible Components, Dice, Fantasy, Fighting","Mike Elliott, Eric M. Lang",2.1636
1359,1359,205046,Capital Lux,2,4,30,30,0,2016,7.12118,6.24605,900,10,"Area Control / Area Influence, Card Drafting, ...",1547,"Bluffing, Card Game, Science Fiction","Eilif Svensson, Kristian Amundsen Østby",2.0000
1478,1478,193308,Spyfall 2,3,12,10,10,0,2017,7.27587,6.19437,712,13,"Acting, Memory, Role Playing, Voting",2065,"Bluffing, Deduction, Party Game, Spies/Secret ...",Alexandr Ushan,1.1667
1802,1802,162559,Smash Up: Munchkin,2,4,45,45,0,2015,6.81948,6.07449,884,12,"Area Control / Area Influence, Hand Management...",3566,"Card Game, Fantasy, Humor",none,2.0952
1834,1834,206931,Noch mal!,1,6,20,20,0,2016,6.97665,6.06526,659,8,Dice Rolling,1123,Dice,"Inka Brand, Markus Brand",1.2143
1959,1959,174614,Apotheca,1,4,30,30,0,2016,6.93168,6.02512,733,13,"Action Point Allowance System, Grid Movement, ...",1583,"Abstract Strategy, Bluffing, Deduction, Puzzle",Andrew Federspiel,1.9524
2164,2165,18291,Unpublished Prototype,0,0,0,0,0,0,6.97370,5.96984,577,0,none,881,none,(Uncredited),2.4000
2457,2458,180602,Game of Trains,2,4,30,30,0,2015,6.39184,5.90795,988,8,Pattern Building,1607,"Card Game, Puzzle, Trains","Alexey Konnov, Alexey Paltsev, Anatoliy Shklyarov",1.2273
2475,2476,23953,Outside the Scope of BGG,0,0,0,0,0,0,6.73655,5.90572,516,0,none,2190,none,(Uncredited),1.6582


In [88]:
# Show value table of max_players
bgg.max_players.value_counts()

4     957
5     583
6     467
2     368
8     126
7      57
10     46
3      26
12     22
1      18
99      9
18      8
16      8
9       6
20      5
0       3
15      3
30      2
24      2
68      1
13      1
11      1
75      1
21      1
Name: max_players, dtype: int64

In [89]:
# Show rows where min_players equals 0
bgg[bgg['min_players']==0]

,rank,game_id,names,min_players,max_players,avg_time,min_time,max_time,year,avg_rating,geek_rating,num_votes,age,mechanic,owned,category,designer,weight
2164,2165,18291,Unpublished Prototype,0,0,0,0,0,0,6.97370,5.96984,577,0,none,881,none,(Uncredited),2.4000
2475,2476,23953,Outside the Scope of BGG,0,0,0,0,0,0,6.73655,5.90572,516,0,none,2190,none,(Uncredited),1.6582
2523,2524,21804,Traditional Card Games,0,0,0,0,0,0,6.52588,5.89507,689,0,none,1203,"Card Game, Game System",(Uncredited),2.0169


In [90]:
# Show value table of min_players
bgg.min_players.value_counts()

2    1861
3     397
1     385
4      58
5       9
8       4
6       4
0       3
Name: min_players, dtype: int64

Vemos que en diferentes variables tenemos valores extremos y valores que no concuerdan con el dataset, como por ejemplo que el tiempo medio de una partida sean 27000 minutos.
A continuación, modificamos los valores extremos y los sustituimos por los valores medios de la misma variable.

**NOTA**

Sí podemos tener juegos con duración muy extendida, en caso de ser un "Legacy", en que el juego se va desarrollando en modo campaña y se tardan decenas de partidas en acabar el juego. Este tipo de juegos típicamente no tiene rejugabilidad.

Por otro lado, la columna avg_time es igual a max_time, por lo que la retiraría del data set

In [91]:
# MIN_PLAYERS
# Substitute 0 with min_players mean in min_players column
bgg.min_players.replace(0,1,inplace=True)

#MAX_PLAYERS
# Substitute values higher than 10 with 10 in max_players column
bgg['max_players'].where(bgg['max_players'] < 10, 10, inplace=True)
bgg.max_players.replace(0,1,inplace=True)


#MIN_TIME
bgg['min_time'].where(bgg['min_time'] < 91, 90, inplace=True)
bgg['min_time'].where(bgg['min_time'] > 16, 15, inplace=True)
bgg.min_time.replace(16,15,inplace=True)
bgg.min_time.replace(42,40,inplace=True)

#AVG_TIME

bgg['avg_time'].where(bgg.avg_time.value_counts()==1, bgg.avg_time.mean(), inplace=True)
mask = bgg.avg_time.map(bgg.avg_time.value_counts()) < 5
bgg.avg_time =  bgg.avg_time.mask(mask, bgg.avg_time.mean())
bgg.avg_time.replace(0,15,inplace=True)

#MAX_TIME
bgg['max_time'].where(bgg['max_time'] < 300, 90, inplace=True)
bgg['max_time'].where(bgg['max_time'] > 10, 10, inplace=True)
bgg['max_time'].where(bgg.max_time.value_counts()==1, bgg.max_time.mean(), inplace=True)
mask = bgg.max_time.map(bgg.max_time.value_counts()) < 5
bgg.max_time =  bgg.max_time.mask(mask, bgg.max_time.mean())
bgg.max_time.replace(0,15,inplace=True)


#YEAR
bgg['year'].where(bgg['year'] > 1950, 1950, inplace=True)
bgg.year.value_counts()


2015    207
2014    195
2013    180
2012    178
2016    176
2011    157
2010    137
2009    134
2008    133
2007    109
2017    105
2004    103
2005    100
2006     94
2003     80
2002     71
1999     52
2001     48
2000     44
1950     37
1998     29
1997     27
1996     25
1995     25
1994     19
1992     19
1986     17
1993     17
1991     15
1985     15
1990     14
1988     13
1983     13
1989     13
1979     12
1977     12
1981     10
1987     10
1982     10
1974      8
1980      8
1984      7
1976      6
1973      5
1975      4
1964      4
1978      4
1962      3
1972      3
1971      2
1967      2
1963      2
1969      2
1960      1
1970      1
1966      1
1965      1
1959      1
1955      1
Name: year, dtype: int64

## 4. Análisis de los datos

*Ideas*
Estudiar la relación entre complejidad (weight) y rating (geek rating y avg rating)

Estudiar la relación entre complejidad (weight) y owned

Estudiar la relación entre rating (geek y avg) y owned

Estudiar qué Mecánicas son las más populares (mechanics vs owned)

Estudiar las categorías más populares (category vs owned)



### 4.x Relación entre Complejidad y Popularidad
Pensamos que la complejidad del juego puede ser un factor clave en cuanto a la popularidad esperada. Analizaremos a continuación la relación entre complejidad (atributo `weight`) y la popularidad (atributo `owned`, esto es, cuántos usuarios han notificado que poseen el juego).

In [92]:
pd.options.plotting.backend = "plotly"
import plotly.express as px

# Filtrar 
#ax1 = bgg.head(250).plot.scatter(x='num_votes', y='weight')



# Zona de test

In [93]:

plot2 = bgg.head(1000).plot.scatter(x="avg_rating", y="weight")
plot2

In [94]:
plot3 = bgg.head(250).plot.scatter(x="owned", y="geek_rating")
plot3

In [95]:
plot4 = bgg.plot.scatter(x="owned", y="num_votes")
plot4

In [96]:
#min_votes = bgg['num_votes'] > 2000
min_weight = bgg['weight'] > 1.5
min_rating = bgg['avg_rating'] > 6

df = bgg[min_weight & min_rating]
fig = px.scatter(df, x='owned', y='num_votes', trendline='ols', log_x=True, log_y=True, color='avg_rating')
#plot4 = bgg[min_votes].plot.scatter(log_x=True, log_y=True, x="owned", y="num_votes", trendline='ols')
#plot4


In [97]:
fig.show()

a = px.get_trendline_results(fig).px_fit_results.iloc[0].rsquared
print(a)


0.9691769629828618


In [98]:
min_votes = bgg['num_votes'] > 5000

In [99]:
plot5 = bgg[min_votes].plot.scatter(x="weight", y="num_votes", text="game_id")
plot5

In [100]:
fig = px.histogram(df, x="weight", y="num_votes", marginal="box",
                   hover_data=df.columns, nbins=30)
fig.show()

In [101]:
plot7 = bgg[min_votes].plot.distplot(x="avg_rating", y="num_votes")

print(bgg[min_votes].avg_rating.mean())
plot7

AttributeError: 'PlotAccessor' object has no attribute 'distplot'

In [ ]:
#plot6 = bgg[min_votes].plot.scatter(x="num_votes", y="geek_rating")

#Çplot6

fig = px.histogram(df, x="avg_rating", y="num_votes", marginal="box",
                   hover_data=df.columns)
fig.show()

NameError: name 'px' is not defined

In [ ]:
import pandas as pd

# Assuming that dataframes df1 and df2 are already defined:

print( "Dataframe 2:")
display(HTML(bgg.head().to_html()))


#bgg=pd.read_csv('../csv/bgg_db_2018_01.csv',sep=',',encoding='latin-1')


### 4.x Relación entre Categorías y Juegos Comprados 
Queremos ver cúales son las temáticas o categorías que mas juegos hay en el mercado, para de este modo saber las tendencias tanto actuales como pasadas.

In [102]:
# Plot top 9 category values count in the dataframe
bgg.category.value_counts().head(9).plot(kind='barh')


Vemos que de los top 9 categorías, 4 son juegos de mesa de guerra. Por otro lado, hay juegos de cartas, dados y estrategia. Ahora, queremos ver si estos juegos han llegado a tener muchas ventas o si son solo ha sido una moda o hay un trend que sean solo juegos de mesa sobre guerras:


In [103]:
# Merge top 9 category column values with owned column values to get the top 9 categories with the most games owned by users in the dataframe and plot it
bgg[['category','owned']].groupby('category').sum().sort_values(by='owned', ascending=False).head(9).plot(kind='barh')


Podemos ver que, por muchos juegos de la categoría 'wargame' existan en el mercado, no indica que sean de los más vendidos a excepción de los juegos de la segunda guerra mundial. No obstante, es la tercera categoría más comprada. Por otro lado, podemos ver que, estas nueve categorías han tenido al menos 100.000 ventas, por lo que puede ser un buen comienzo para saber por qué temática comenzar, llegando a ver que los tipos de cartas han llegado hasta las casi 350 mil copias vendidas. 

### 4.x Relación entre Mecánicas de Juegos y Juegos Comprados 
Queremos ver cúales son las temáticas o categorías que mas juegos hay en el mercado, para de este modo saber las tendencias tanto actuales como pasadas.

In [104]:
# Plot top 9 category values count in the dataframe
bgg.mechanic.value_counts().head(9).plot(kind='barh')

In [106]:
# Merge top 9 category column values with owned column values to get the top 9 categories with the most games owned by users in the dataframe and plot it
bgg[['mechanic','owned']].groupby('mechanic').sum().sort_values(by='owned', ascending=False).head(9).plot(kind='barh')

## Referencias consultadas


Board Game Rank \[en línea\] \[fecha de consulta: 30 de mayo de 2022\]. Disponible en: https://boardgamegeek.com/browse/boardgame?sort=rank&sortdir=desc

BoardGameGeek FAQ \[en línea\] \[fecha de consulta: 31 de mayo de 2022\]. Disponible en: https://boardgamegeek.com/wiki/page/BoardGameGeek_FAQ#toc13

BoardGameWiki. Weight \[en línea\] \[fecha de consulta: 31 de mayo de 2022\]. Disponible en: https://boardgamegeek.com/wiki/page/Weight

How to delete a column in pandas \[en línea\] \[fecha de consulta: 01 de junio de 2022\]. Disponible en: https://www.educative.io/edpresso/how-to-delete-a-column-in-pandas

Pandas Plotting Backend in Python \[en línea\] \[fecha de consulta: 02 de junio de 2022\]. Disponible en: https://plotly.com/python/pandas-backend/

Working with Markdown tables in GitHub \[en línea\] \[fecha de consulta: 03 de junio de 2022\]. Disponible en: https://www.pluralsight.com/guides/working-tables-github-markdown

Practical Business Python. Overview of Pandas Data Types \[en línea\] \[fecha de consulta: 3 de junio de 2022\]. Disponible en: https://pbpython.com/pandas_dtypes.html

